# pET19K_Haggfish Alpha Protein Model Creation

## 1. Load the python packages

In [1]:
from __future__ import print_function, division, absolute_import

# python imports
import re
from os.path import join
from collections import defaultdict
import pickle
import pandas as pd

# third party imports
import pandas
import tabulate
import cobra
pd.set_option('display.max_columns', 100)
pd.set_option('display.width',100)
pd.set_option('display.max_colwidth',300)

# ECOLIme
import ecolime
from ecolime import (transcription, translation, flat_files, generics, formulas, compartments)

# COBRAme
import cobrame
from cobrame.util import building, mu, me_model_interface
#from cobrame.io.json import save_json_me_model, save_reduced_json_me_model

/usr/lib/python3.6/site-packages/cobra-0.5.11-py3.6-linux-x86_64.egg/cobra/solvers/__init__.py:63: UserWarning: No LP solvers found
  warn("No LP solvers found")
/usr/lib/python3.6/site-packages/cobra-0.5.11-py3.6-linux-x86_64.egg/cobra/io/sbml3.py:24: UserWarning: Install lxml for faster SBML I/O
  warn("Install lxml for faster SBML I/O")
/usr/lib/python3.6/site-packages/cobra-0.5.11-py3.6-linux-x86_64.egg/cobra/io/__init__.py:12: UserWarning: cobra.io.sbml requires libsbml
  warn("cobra.io.sbml requires libsbml")


## 2. Load the iJL1678b model

Load the iJL1678b model in pickle format that is located in the current directory.

In [2]:
with open('iJL1678b.pickle', 'rb') as f:
    me = pickle.load(f)

## 3. Define ME-Model Solve Functions

In [3]:
def solve_me_model(me, max_mu, precision=1e-6, min_mu=0, using_soplex=False,
                  compiled_expressions=None):
    if using_soplex:
        from cobrame.solve.algorithms import binary_search
        binary_search(me, min_mu=min_mu, max_mu=max_mu, debug=True, mu_accuracy=precision,
                      compiled_expressions=compiled_expressions)
    else:
        from qminospy.me1 import ME_NLP1
        # The object containing solveME methods--composite that uses a ME model object 
        me_nlp = ME_NLP1(me, growth_key='mu')
        # Use bisection for now (until the NLP formulation is worked out)
        muopt, hs, xopt, cache = me_nlp.bisectmu(precision=precision, mumax=max_mu)
        me.solution.f = me.solution.x_dict['biomass_dilution']
        
def show_escher_map(me, solution=None):
    import escher
    view = escher.Builder("iJO1366.Central metabolism")
    view.reaction_data = me.get_metabolic_flux(solution=solution)
    return view

## 4. Create the Alpha Transcription Reaction

Complete the steps necessary to create the Alpha Transcription Reaction

In [4]:
locus_id = 'Alpha'
left_pos =  None
right_pos = None
seq = 'ATGAGCATCAGCCAGACCGTTAGCAAAAGCTATACCAAAAGCGTTAGCCGTGGTGGTCAGGGTGTTAGCTATAGCCAGAGCAGCAGCCATAAAGTTGGTGGTGGTAGCGTTCGTTATGGCACCACCTATAGCAGCGGTGGTATTAGCCGTGTTCTGGGTTTTCAAGGTGGTGCAGGCGGTGCAGCAAGCGCAGGTTTTGGTGGTAGTGTTGGTGGTTCAGGTCTGAGCCGTGTGTTAGGTGGTAGCATGGTTAGCGGTTATCGTAGCGGTATGGGTGTTGGCGGTCTGAGTCTGAGCGGCACCGCAGGTCTGCCGGTTAGCCTGCGTGGTGTTGGTGCAGGTAAAGCACTGCATGCAATTACCAGCGCATTTCGTACACGCGTTGGTGGTCCGGGTACAAGCGTTGGCGGTTATGGTGTGAATTATAGCTTTCTGCCGAGCACAGCAGGTCCGAGCTTTGGTGGCCCTTTTGGCGGTCCGTTTGGAGGTCCTTTTGGTGGACCATTAGGTCCGGGTTATATTGATCCGGCAACACTGCCGAGTCCGGATACCGTTCAGCATACCCGTATTCGTGAAAAACAGGATCTGCAGACACTGAATACCAAATTTGCCAATCTGGTTGATCAGGTTCGTACCCTGGAACAGCATAATGCAATTCTGAAAGCACAGATTAGCATGATTACCAGTCCGAGCGATACACCGGAAGGTCCGGTTAATACCGCAGTTGTTGCAAGCACCGTTACCGCAACCTATAATGCACAGATTGAAGATCTGCGTACCACCAATACCGCACTGCATAGCGAAATTGATCATCTGACAACCATCATCAACGACATCACCACCAAATATGAAGAACAGGTTGAAGTTACCCGTACACTGGAAACCGATTGGAATACCAACAAAGATAACATCGATAACACCTACCTGACCATTGTTGATCTGCAAACCAAAGTTCAAGGTCTGGATGAGCAGATTAATACCACAAAGCAGATTTATAACGCACGTGTTCGTGAAGTTCAGGCAGCAGTTACCGGTGGTCCGACCGCAGCATATAGCATTCGTGTTGATAATACCCATCAGGCCATTGATCTGACCACCTCACTGCAAGAAATGAAAACACATTATGAAGTGCTGGCAACCAAAAGCCGTGAAGAAGCATTTACCCAGGTTCAGCCTCGTATTCAAGAAATGGCAGTGACCGTGCAGGCAGGTCCGCAGGCAATTATTCAGGCAAAAGAACAAATCCACGTGTTCAAACTGCAGATTGATAGCGTTCATCGTGAAATCGATCGTCTGCATCGTAAAAATACCGATGTGGAACGTGAAATTACCGTGATCGAAACCAATATTCATACCCAGAGTGATGAATGGACCAACAATATTAACAGCCTGAAAGTTGATCTGGAAGTGATCAAAAAGCAGATCACCCAGTATGCACGTGATTATCAAGATCTGCTGGCCACAAAAATGAGCCTGGATGTTGAAATTGCAGCCTACAAAAAACTGCTGGATAGCGAAGAAACCCGTATTAGTCATGGTGGTGGCATTACCATTACAACCAATGCAGGCACCTTTCCTGGTGGTCTGAGCGCAGCCCCAGGTGGTGGTGCAAGCTATGCAATGGTTCCGGCAGGCGTTGGAGGTGTTGGTCTGGCAGGCGTAGGTGGTTATGGTTTTCGTAGCATGGGTGGCGGTGGTGGCGTTGGCTATGGTGCCGGTGGCGGAGGTGTAGGTTATGGCGTCGGTGGTGGTTTTGGAGGTGGTATGGGTATGAGCATGAGCCGTATGAGTATGGGTGCAGCAGTTGGCGGTGGCAGCTATGGTAGCGGCAGCGGTTATAGCGGTGGCTTTGGTCTGAGTAGCAGCCGTGCAGGTTATAGTGCAAGCCGTAAAAGCTATTCAAGCGCACGTAGCAGCAGTCGTATCTAT'
strand = None
rna_type = 'mRNA'

building.create_transcribed_gene(me, locus_id, rna_type, seq)
building.add_transcription_reaction(me, 'TU_Alpha', {'Alpha'}, seq)
print(me.reactions.transcription_TU_Alpha.reaction)
me.reactions.transcription_TU_Alpha.update()

469 atp_c + 549 ctp_c + 411 gtp_c + 500 utp_c --> RNA_Alpha + 614.960167029 mRNA_biomass + 1929 ppi_c


/source/cobrame/cobrame/core/reaction.py:852 UserWarning: RNA Polymerase () not found


###  Add the RNAP to the Alpha Transcription Reaction

In [5]:
me.reactions.transcription_TU_Alpha.transcription_data.RNA_polymerase = 'RNAP70-CPLX'
me.reactions.transcription_TU_Alpha.update()

List the transcription data ("transcription_data") associated with the Alpha transcription reaction ("transcription_TU_Alpha").

In [6]:
rxn = me.reactions.transcription_TU_Alpha
data = rxn.transcription_data
pd.DataFrame({i: str(v) for i, v in data.__dict__.items()}, index=['Attribute Values']).T

,Attribute Values
id,TU_Alpha
_model,iJL1678b-ME
_parent_reactions,{'transcription_TU_Alpha'}
nucleotide_sequence,ATGAGCATCAGCCAGACCGTTAGCAAAAGCTATACCAAAAGCGTTAGCCGTGGTGGTCAGGGTGTTAGCTATAGCCAGAGCAGCAGCCATAAAGTTGGTGGTGGTAGCGTTCGTTATGGCACCACCTATAGCAGCGGTGGTATTAGCCGTGTTCTGGGTTTTCAAGGTGGTGCAGGCGGTGCAGCAAGCGCAGGTTTTGGTGGTAGTGTTGGTGGTTCAGGTCTGAGCCGTGTGTTAGGTGGTAGCATGGTTAGCGGTTATCGTAGCGGTATGGGTGTTGGCGGTCTGAGTCTGAG...
RNA_products,{'RNA_Alpha'}
RNA_polymerase,RNAP70-CPLX
subreactions,"defaultdict(<class 'int'>, {})"


List the attributes of the Alpha transcription reaction.

In [7]:
rxn = me.reactions.transcription_TU_Alpha
data = rxn
pd.DataFrame({i: str(v) for i, v in data.__dict__.items()}, index=['Attribute Values']).T

,Attribute Values
id,transcription_TU_Alpha
name,
notes,{}
annotation,{}
_gene_reaction_rule,
subsystem,
_genes,set()
_metabolites,"{<RNAP RNAP70-CPLX at 0x7f9f0a244390>: -0.00921844765619026*mu - 0.00360902225739849, <TranscribedGene RNA_Alpha at 0x7f9f0a36f630>: 1, <Metabolite utp_c at 0x7f9f0a244dd8>: -500, <Metabolite atp_c at 0x7f9f0a244978>: -469, <Metabolite ctp_c at 0x7f9f0a244eb8>: -549, <Metabolite gtp_c at 0x7f9f0..."
_model,iJL1678b-ME
_objective_coefficient,0.0


List the complete reaction associated with the Alpha transcription reaction.

In [8]:
me.reactions.get_by_id('transcription_TU_Alpha').reaction

'0.00921844765619026*mu + 0.00360902225739849 RNAP70-CPLX + 469 atp_c + 549 ctp_c + 411 gtp_c + 500 utp_c --> RNA_Alpha + 614.960167029 mRNA_biomass + 1929 ppi_c'

List the RNAP associated with the Alpha transcription reaction

In [9]:
me.reactions.transcription_TU_Alpha.transcription_data.RNA_polymerase

'RNAP70-CPLX'

## 5. Create the Alpha Translation Reaction

Complete all the steps necessary to create the Alpha Translation Reaction

### Storing information about a translation reaction

In [10]:
data = cobrame.TranslationData('Alpha', me, 'RNA_Alpha', 'protein_Alpha')
data.nucleotide_sequence = seq

### Create translation reaction

By associating the TranslationReaction with its corresponding TranslationData object and running the update function, COBRAme will create a translation reaction for the nucleotide sequence given based on the organisms codon table and prespecified translation machinery.

In [11]:
rxn = cobrame.TranslationReaction('translation_Alpha')
rxn.translation_data = data
me.add_reaction(rxn)

### Updating the translation reaction

Note that this process does not include the ribosome in the reaction.

In [12]:
rxn.update()
print(rxn.reaction)

0.000498399634202103*mu + 0.000195123456790123 + 0.00598079561042524*(mu + 0.3915)/mu RNA_Alpha + 2.55589556001079e-8*mu + 1.00063311174422e-8 RNA_degradosome + 48 ala__L_c + 30 arg__L_c + 21 asn__L_c + 24 asp__L_c + 2.88274348422497*(mu + 0.3915)/mu atp_c + 31 gln__L_c + 26 glu__L_c + 102 gly_c + 13 his__L_c + 36 ile__L_c + 38 leu__L_c + 23 lys__L_c + 3.67795106755342*(mu + 0.3915)/mu mRNA_biomass + 14 met__L_c + 15 phe__L_c + 21 pro__L_c + 0.00921844765619026*mu + 0.00360902225739849 ribosome + 66 ser__L_c + 58 thr__L_c + 2 trp__L_c + 26 tyr__L_c + 49 val__L_c --> 2.88274348422497*(mu + 0.3915)/mu adp_c + 2.80499314128944*(mu + 0.3915)/mu amp_c + 3.28345679012346*(mu + 0.3915)/mu cmp_c + 2.45810699588477*(mu + 0.3915)/mu gmp_c + 642.0 - 2.88274348422497*(mu + 0.3915)/mu h2o_c + 2.88274348422497*(mu + 0.3915)/mu h_c + 2.88274348422497*(mu + 0.3915)/mu pi_c + protein_Alpha + 66.69648898000001 protein_biomass + 2.99039780521262*(mu + 0.3915)/mu ump_c


### Incorporate the ribosome

Add the ribosome to the reaction.

In [13]:
ribosome = cobrame.Ribosome('ribosome')
me.add_metabolites([ribosome])
me.reactions.translation_Alpha.update()
print(me.reactions.translation_Alpha.reaction)

0.000498399634202103*mu + 0.000195123456790123 + 0.00598079561042524*(mu + 0.3915)/mu RNA_Alpha + 2.55589556001079e-8*mu + 1.00063311174422e-8 RNA_degradosome + 48 ala__L_c + 30 arg__L_c + 21 asn__L_c + 24 asp__L_c + 2.88274348422497*(mu + 0.3915)/mu atp_c + 31 gln__L_c + 26 glu__L_c + 102 gly_c + 13 his__L_c + 36 ile__L_c + 38 leu__L_c + 23 lys__L_c + 3.67795106755342*(mu + 0.3915)/mu mRNA_biomass + 14 met__L_c + 15 phe__L_c + 21 pro__L_c + 0.00921844765619026*mu + 0.00360902225739849 ribosome + 66 ser__L_c + 58 thr__L_c + 2 trp__L_c + 26 tyr__L_c + 49 val__L_c --> 2.88274348422497*(mu + 0.3915)/mu adp_c + 2.80499314128944*(mu + 0.3915)/mu amp_c + 3.28345679012346*(mu + 0.3915)/mu cmp_c + 2.45810699588477*(mu + 0.3915)/mu gmp_c + 642.0 - 2.88274348422497*(mu + 0.3915)/mu h2o_c + 2.88274348422497*(mu + 0.3915)/mu h_c + 2.88274348422497*(mu + 0.3915)/mu pi_c + protein_Alpha + 66.69648898000001 protein_biomass + 2.99039780521262*(mu + 0.3915)/mu ump_c


### Add Subreactions to the Alpha model

Create the subreactions that represent elongation, initiation and termination

In [14]:
elongation = ['FusA_mono_elongation', 'Tuf_gtp_regeneration']
initiation = ['Translation_gtp_initiation_factor_InfB', 'Translation_initiation_factor_InfA', 
              'Translation_initiation_factor_InfC', 'fmet_addition_at_START', 'peptide_deformylase_processing', 
              'peptide_chain_release', 'ribosome_recycler']
termination = {'UGA':'PrfB_mono'}
me.process_data.get_by_id('Alpha').add_elongation_subreactions(elongation_subreactions=set(elongation))
me.process_data.get_by_id('Alpha').add_initiation_subreactions(start_codons=set(['AUG']), start_subreactions=set(initiation))
me.process_data.get_by_id('Alpha').add_termination_subreactions(translation_terminator_dict=termination)
#me.reactions.translation_Alpha.update(verbose=False)

/source/cobrame/cobrame/core/processdata.py:896 UserWarning: No termination enzyme for RNA_Alpha


List the subreactions associated with Alpha translation reaction stored in the ProcessData.

In [15]:
me.process_data.get_by_id('Alpha').subreactions

defaultdict(int,
            {'FusA_mono_elongation': 642.0,
             'Tuf_gtp_regeneration': 642.0,
             'met_addition_at_AUG': 13,
             'ser_addition_at_AGC': 53,
             'ile_addition_at_AUC': 11,
             'gln_addition_at_CAG': 24,
             'thr_addition_at_ACC': 46,
             'val_addition_at_GUU': 37,
             'lys_addition_at_AAA': 21,
             'tyr_addition_at_UAU': 23,
             'arg_addition_at_CGU': 29,
             'gly_addition_at_GGU': 77,
             'his_addition_at_CAU': 12,
             'gly_addition_at_GGC': 20,
             'ile_addition_at_AUU': 25,
             'leu_addition_at_CUG': 36,
             'phe_addition_at_UUU': 14,
             'gln_addition_at_CAA': 7,
             'ala_addition_at_GCA': 42,
             'ser_addition_at_AGU': 10,
             'ser_addition_at_UCA': 3,
             'val_addition_at_GUG': 9,
             'leu_addition_at_UUA': 2,
             'pro_addition_at_CCG': 15,
             'thr_a

List the subreactions generated from the nucleatide sequence that are stored in the ProcessData.

In [16]:
me.process_data.get_by_id('Alpha').subreactions_from_sequence

{'met_addition_at_AUG': 13,
 'ser_addition_at_AGC': 53,
 'ile_addition_at_AUC': 11,
 'gln_addition_at_CAG': 24,
 'thr_addition_at_ACC': 46,
 'val_addition_at_GUU': 37,
 'lys_addition_at_AAA': 21,
 'tyr_addition_at_UAU': 23,
 'arg_addition_at_CGU': 29,
 'gly_addition_at_GGU': 77,
 'his_addition_at_CAU': 12,
 'gly_addition_at_GGC': 20,
 'ile_addition_at_AUU': 25,
 'leu_addition_at_CUG': 36,
 'phe_addition_at_UUU': 14,
 'gln_addition_at_CAA': 7,
 'ala_addition_at_GCA': 42,
 'ser_addition_at_AGU': 10,
 'ser_addition_at_UCA': 3,
 'val_addition_at_GUG': 9,
 'leu_addition_at_UUA': 2,
 'pro_addition_at_CCG': 15,
 'thr_addition_at_ACA': 12,
 'arg_addition_at_CGC': 1,
 'asn_addition_at_AAU': 14,
 'pro_addition_at_CCU': 4,
 'gly_addition_at_GGA': 5,
 'pro_addition_at_CCA': 2,
 'asp_addition_at_GAU': 23,
 'glu_addition_at_GAA': 25,
 'ala_addition_at_GCC': 6,
 'asn_addition_at_AAC': 7,
 'asp_addition_at_GAC': 1,
 'trp_addition_at_UGG': 2,
 'tyr_addition_at_UAC': 2,
 'glu_addition_at_GAG': 1,
 'lys_

### Updating the Alpha translation reaction

In [17]:
translation_subreactions = me.process_data.Alpha.subreactions_from_sequence
print("Before adding tRNA subreaction")
print("------------------------------")
print(me.reactions.translation_Alpha.reaction)
print("")
# Link tranlation_data to subreactions and update
for subreaction, value in translation_subreactions.items():
    me.process_data.Alpha.subreactions[subreaction] = value
me.reactions.translation_Alpha.update(verbose=False)
print("After adding tRNA subreaction")
print("-----------------------------")
print(me.reactions.translation_Alpha.reaction)

Before adding tRNA subreaction
------------------------------
0.000498399634202103*mu + 0.000195123456790123 + 0.00598079561042524*(mu + 0.3915)/mu RNA_Alpha + 2.55589556001079e-8*mu + 1.00063311174422e-8 RNA_degradosome + 48 ala__L_c + 30 arg__L_c + 21 asn__L_c + 24 asp__L_c + 2.88274348422497*(mu + 0.3915)/mu atp_c + 31 gln__L_c + 26 glu__L_c + 102 gly_c + 13 his__L_c + 36 ile__L_c + 38 leu__L_c + 23 lys__L_c + 3.67795106755342*(mu + 0.3915)/mu mRNA_biomass + 14 met__L_c + 15 phe__L_c + 21 pro__L_c + 0.00921844765619026*mu + 0.00360902225739849 ribosome + 66 ser__L_c + 58 thr__L_c + 2 trp__L_c + 26 tyr__L_c + 49 val__L_c --> 2.88274348422497*(mu + 0.3915)/mu adp_c + 2.80499314128944*(mu + 0.3915)/mu amp_c + 3.28345679012346*(mu + 0.3915)/mu cmp_c + 2.45810699588477*(mu + 0.3915)/mu gmp_c + 642.0 - 2.88274348422497*(mu + 0.3915)/mu h2o_c + 2.88274348422497*(mu + 0.3915)/mu h_c + 2.88274348422497*(mu + 0.3915)/mu pi_c + protein_Alpha + 66.69648898000001 protein_biomass + 2.990397805212

/source/cobrame/cobrame/core/processdata.py:256 UserWarning: met (generic_tRNA_GAG_glu__L_c) does not have formula


List the reactants associated with the Alpha translation reaction. In this list all the subreactions should be listed.

In [18]:
me.reactions.translation_Alpha.reactants

[<Metabolite met__L_c at 0x7f9f09547978>,
 <Metabolite h2o_c at 0x7f9f0a244a20>,
 <Metabolite ser__L_c at 0x7f9f09547eb8>,
 <Metabolite ile__L_c at 0x7f9f09553240>,
 <Metabolite gln__L_c at 0x7f9f09553278>,
 <Metabolite thr__L_c at 0x7f9f09547e10>,
 <Metabolite val__L_c at 0x7f9f09547b70>,
 <Metabolite lys__L_c at 0x7f9f09547c88>,
 <Metabolite tyr__L_c at 0x7f9f09553470>,
 <Metabolite arg__L_c at 0x7f9f09547a90>,
 <Metabolite gly_c at 0x7f9f09547cf8>,
 <Metabolite his__L_c at 0x7f9f09553390>,
 <Metabolite leu__L_c at 0x7f9f09547b38>,
 <Metabolite phe__L_c at 0x7f9f09547cc0>,
 <Metabolite ala__L_c at 0x7f9f09547f60>,
 <Metabolite pro__L_c at 0x7f9f095532e8>,
 <Metabolite asn__L_c at 0x7f9f09553080>,
 <Metabolite asp__L_c at 0x7f9f09553160>,
 <Metabolite glu__L_c at 0x7f9f095530b8>,
 <Metabolite trp__L_c at 0x7f9f095534e0>,
 <Ribosome ribosome at 0x7f9f09553550>,
 <TranscribedGene RNA_Alpha at 0x7f9f0a36f630>,
 <Metabolite atp_c at 0x7f9f0a244978>,
 <Complex RNA_degradosome at 0x7f9f0a26

This completes the creation of the translation reaction

## 6. Create Demand Reaction for Alpha

In [19]:
new_reaction = cobrame.MEReaction('DM_protein_Alpha')
me.add_reaction(new_reaction)

In [20]:
me.reactions.get_by_id('DM_protein_Alpha').reaction = 'protein_Alpha ->'
me.reactions.get_by_id('DM_protein_Alpha').reaction

'protein_Alpha --> '

In [21]:
rxn = me.reactions.DM_protein_Alpha
data = rxn
pd.DataFrame({i: str(v) for i, v in data.__dict__.items()}, index=['Attribute Values']).T

,Attribute Values
id,DM_protein_Alpha
name,
notes,{}
annotation,{}
_gene_reaction_rule,
subsystem,
_genes,set()
_metabolites,{<TranslatedGene protein_Alpha at 0x7f9f0a32cd68>: -1}
_model,iJL1678b-ME
_objective_coefficient,0.0


### Find the amino acid count

In [22]:
me.process_data.Alpha.amino_acid_count

defaultdict(int,
            {'met__L_c': 14,
             'ser__L_c': 66,
             'ile__L_c': 36,
             'gln__L_c': 31,
             'thr__L_c': 58,
             'val__L_c': 49,
             'lys__L_c': 23,
             'tyr__L_c': 26,
             'arg__L_c': 30,
             'gly_c': 102,
             'his__L_c': 13,
             'leu__L_c': 38,
             'phe__L_c': 15,
             'ala__L_c': 48,
             'pro__L_c': 21,
             'asn__L_c': 21,
             'asp__L_c': 24,
             'glu__L_c': 26,
             'trp__L_c': 2})

In [23]:
import pickle
with open('iJL1678b_pET19K_HF_alpha.pickle', 'wb') as f:
    pickle.dump(me, f)

In [24]:
with open('iJL1678b_pET19K_HF_alpha.pickle', 'rb') as f:
    test_me = pickle.load(f)

In [25]:
test_me

<MEModel iJL1678b-ME at 0x7f9f0a005780>

In [26]:
test_me.global_info

{'kt': 4.5,
 'r0': 0.087,
 'k_deg': 12.0,
 'm_rr': 1453.0,
 'm_aa': 0.109,
 'f_rRNA': 0.86,
 'm_nt': 0.324,
 'f_mRNA': 0.02,
 'm_tRNA': 25.0,
 'f_tRNA': 0.12,
 'temperature': 37,
 'propensity_scaling': 0.45,
 'GC_fraction': 0.507896997096}